#GitHub Assignment


In [ ]:
import re

def optimize_code(code_lines):
    instructions = []
    values = {}

    for line in code_lines:
        parts = re.match(r"(\w+)\s*=\s*(\w+)\s*([*+])\s*(\w+)", line)
        var, op1, op, op2 = parts.groups()

        op1_val = values.get(op1, op1)
        op2_val = values.get(op2, op2)

        try:
            val1 = int(op1_val)
            val2 = int(op2_val)
            if op == '*':
                result = val1 * val2
            else:
                result = val1 + val2

            instructions.append({'var': var, 'is_const': True, 'val': result})
            values[var] = result
            continue
        except (ValueError, TypeError):
            pass

        if op == '*' and op2_val == '1':
            instructions.append({'var': var, 'is_const': False, 'val': op1})
            values[var] = values.get(op1, op1)
            continue
        if op == '+' and op2_val == '0':
            instructions.append({'var': var, 'is_const': False, 'val': op1})
            values[var] = values.get(op1, op1)
            continue

        instructions.append({'var': var, 'is_const': False, 'val': (op1, op, op2)})

    propagated_instructions = []
    for instr in instructions:
        if not instr['is_const']:
            if isinstance(instr['val'], str) and instr['val'] in values:
                new_val = values[instr['val']]
                if new_val != instr['val']:
                     instr['val'] = new_val
        propagated_instructions.append(instr)
    instructions = propagated_instructions

    if not instructions:
        return []

    live_vars = {instructions[-1]['var']}
    final_code = []

    for instr in reversed(instructions):
        var = instr['var']
        if var in live_vars:
            live_vars.remove(var)
            final_code.append(instr)
            if not instr['is_const'] and isinstance(instr['val'], str):
                live_vars.add(instr['val'])
        else:
            pass

    final_code.reverse()
    return final_code

if __name__ == "__main__":
    input_code = [
        "x = 2 * 8",
        "y = x * 1",
        "z = y + 0",
    ]

    optimized_code = optimize_code(input_code)

    for instr in optimized_code:
        print(f"{instr['var']} = {instr['val']}")


In [2]:
def infix_to_postfix(expression):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2}
    output = []
    operators = []

    # A simple tokenizer; assumes single-character variables and operators
    import re
    tokens = re.findall(r"(\b\w+\b|[+\-*/()])", expression)

    for token in tokens:
        if token.isalnum():
            output.append(token)
        elif token == '(':
            operators.append(token)
        elif token == ')':
            while operators and operators[-1] != '(':
                output.append(operators.pop())
            operators.pop()
        else:
            while (operators and operators[-1] != '(' and
                   precedence.get(operators[-1], 0) >= precedence.get(token, 0)):
                output.append(operators.pop())
            operators.append(token)

    while operators:
        output.append(operators.pop())

    return output

def generate_assembly(postfix_tokens):
    operator_map = {'+': 'ADD', '-': 'SUB', '*': 'MUL', '/': 'DIV'}
    for token in postfix_tokens:
        if token in operator_map:
            print(operator_map[token])
        else:
            print(f"PUSH {token}")

if __name__ == "__main__":
    input_expression = "(a+b)*c"

    postfix = infix_to_postfix(input_expression)
    generate_assembly(postfix)


PUSH a
PUSH b
ADD
PUSH c
MUL


#Special Assignment for Fast Learner


In [3]:
def shunt(infix):
    infix = list(infix)[::-1]
    specials = {'*': 50, '.': 40, '|': 30}
    pofix = ""
    stack = ""

    # Add explicit concatenation operators
    processed_infix = ""
    for i in range(len(infix) - 1, -1, -1):
        processed_infix += infix[i]
        if i > 0:
            c1 = infix[i]
            c2 = infix[i-1]
            if (c1.isalnum() or c1 == ')' or c1 == '*') and \
               (c2.isalnum() or c2 == '(' or c2 == '.'): # Added c2 == '.' to handle cases like a.(b|c)
                processed_infix += '.'

    infix = list(processed_infix)[::-1]

    while infix:
        c1 = infix.pop()

        if c1 == '(':
            stack += c1
        elif c1 == ')':
            while stack and stack[-1] != '(':
                pofix += stack[-1]
                stack = stack[:-1]
            if stack and stack[-1] == '(':
                stack = stack[:-1] # Pop '('
            else:
                raise ValueError("Mismatched parentheses")
        elif c1 in specials:
            while stack and stack[-1] in specials and specials[c1] <= specials[stack[-1]]:
                pofix += stack[-1]
                stack = stack[:-1]
            stack += c1
        else:
            pofix += c1

    while stack:
        if stack[-1] == '(':
            raise ValueError("Mismatched parentheses")
        pofix += stack[-1]
        stack = stack[:-1]

    return pofix

class State:
    def __init__(self, label=None, edges=None):
        self.edges = edges if edges else []
        self.label = label

class NFA:
    def __init__(self, start, end):
        self.start = start
        self.end = end

def thompson(postfix):
    nfa_stack = []

    for c in postfix:
        if c == '.':
            # Concatenation
            if len(nfa_stack) < 2:
                raise ValueError("Invalid postfix expression: Not enough fragments for concatenation")
            frag2 = nfa_stack.pop()
            frag1 = nfa_stack.pop()
            frag1.end.edges.append(frag2.start)
            new_frag = NFA(frag1.start, frag2.end)
            nfa_stack.append(new_frag)
        elif c == '|':
            # Union
            if len(nfa_stack) < 2:
                 raise ValueError("Invalid postfix expression: Not enough fragments for union")
            frag2 = nfa_stack.pop()
            frag1 = nfa_stack.pop()
            start = State()
            start.edges.append(frag1.start)
            start.edges.append(frag2.start)
            end = State()
            frag1.end.edges.append(end)
            frag2.end.edges.append(end)
            new_frag = NFA(start, end)
            nfa_stack.append(new_frag)
        elif c == '*':
            # Kleene star
            if not nfa_stack:
                 raise ValueError("Invalid postfix expression: Not enough fragments for Kleene star")
            frag = nfa_stack.pop()
            start = State()
            end = State()
            start.edges.append(frag.start)
            start.edges.append(end)
            frag.end.edges.append(frag.start)
            frag.end.edges.append(end)
            new_frag = NFA(start, end)
            nfa_stack.append(new_frag)
        else:
            # Literal character
            end = State()
            start = State(label=c, edges=[end])
            new_frag = NFA(start, end)
            nfa_stack.append(new_frag)

    if len(nfa_stack) != 1:
        raise ValueError("Invalid postfix expression: Stack does not contain a single NFA at the end")

    return nfa_stack.pop()

def subset_construction(nfa, alphabet):
    dfa_states = []
    dfa_transitions = {}

    start_closure = follow_epsilons(nfa.start)
    dfa_states.append(start_closure)

    queue = [start_closure]

    while queue:
        current_states = queue.pop(0)
        dfa_transitions[tuple(sorted(id(s) for s in current_states))] = {}

        for symbol in alphabet:
            move_states = set()
            for nfa_state in current_states:
                if nfa_state.label == symbol:
                    for edge in nfa_state.edges:
                         move_states.add(edge)

            if not move_states:
                continue

            next_states = set()
            for s in move_states:
                next_states.update(follow_epsilons(s))

            if next_states not in dfa_states:
                dfa_states.append(next_states)
                queue.append(next_states)

            current_id = tuple(sorted(id(s) for s in current_states))
            next_id = tuple(sorted(id(s) for s in next_states))
            dfa_transitions[current_id][symbol] = next_id

    final_states = [s for s in dfa_states if nfa.end in s]

    return dfa_states, dfa_transitions, final_states

def follow_epsilons(state, visited=None):
    if visited is None:
        visited = set()

    states = {state}
    visited.add(state)

    if state.label is None:
        for edge in state.edges:
            if edge not in visited:
                states.update(follow_epsilons(edge, visited))

    return states

def print_dfa_table(dfa_states, dfa_transitions, final_states, alphabet):
    state_map = {tuple(sorted(id(s) for s in states)): f"S{i}" for i, states in enumerate(dfa_states)}

    header = "{:<10}".format("State")
    for symbol in alphabet:
        header += "{:<10}".format(symbol)
    print(header)
    print("-" * len(header))

    for states, state_name in state_map.items():
        is_final = ""
        # Check if this is the start state (the first state in dfa_states)
        if tuple(sorted(id(s) for s in dfa_states[0])) == states:
             is_final += " (Start)"

        # Check if this is a final state
        for s_set in final_states:
             if tuple(sorted(id(s) for s in s_set)) == states:
                 is_final = "(Final)"
                 break


        row = "{:<10}".format(state_name + " " + is_final.strip())

        transitions = dfa_transitions.get(states, {})
        for symbol in alphabet:
            dest_state_ids = transitions.get(symbol)
            if dest_state_ids:
                row += "{:<10}".format(state_map.get(dest_state_ids, "-"))
            else:
                row += "{:<10}".format("-")
        print(row)


if __name__ == "__main__":
    regex = "(a|b)*abb"
    alphabet = sorted(list(set(c for c in regex if c.isalnum())))

    postfix_regex = shunt(regex)
    print(f"Postfix regex: {postfix_regex}") # Print postfix regex for debugging

    nfa = thompson(postfix_regex)
    dfa_states, dfa_transitions, final_states = subset_construction(nfa, alphabet)

    print(f"\nDFA for: {regex}\n")
    print_dfa_table(dfa_states, dfa_transitions, final_states, alphabet)

Postfix regex: ab|*a.b.b.

DFA for: (a|b)*abb

State     a         b         
------------------------------
S0 (Start)S1        S2        
S1        S1        S3        
S2        S1        S2        
S3        S1        S4        
S4 (Final)S1        S2        


In [5]:
def dfa(string):
    state = "q0"
    for ch in string:
        if state == "q0":
            if ch == "0":
                state = "q1"
            elif ch == "1":
                state = "q0"
            else:
                state = "q3"
        elif state == "q1":
            if ch == "0":
                state = "q1"
            elif ch == "1":
                state = "q2"
            else:
                state = "q3"
        elif state == "q2":
            if ch == "0":
                state = "q1"
            elif ch == "1":
                state = "q0"
            else:
                state = "q3"
        else:
            state = "q3"
    return state == "q2"

test_strings = ["1101", "111", "0001"]

for s in test_strings:
    if dfa(s):
        print(f"{s} → Accepted")
    else:
        print(f"{s} → Rejected")


1101 → Accepted
111 → Rejected
0001 → Accepted
